In [1]:
import numpy as np
from pathlib import Path 

In [2]:
def data_preprocessing(path):
    
    with path.open(mode="r", encoding="utf-8") as file:
        
        data = file.read().splitlines()
        data = [list(line) for line in data]
        data = np.array(data)

    return data

In [3]:
def find_init_pos(data):
    
    for i,row in enumerate(data):
        
        for j, spot in enumerate(row):
            
            if spot == "S":
                
                init_pos = (i,j)
                return init_pos 

In [4]:
def move_validation(move_dict, sub_problems, size):
    
    
    valid_move = []
    oposite_move = {"up":"down", "down":"up", "right":"left", "left":"right"}
    
    for key, value in move_dict.items():
        
        if(value[0] >=0 and value[1] >= 0 and value[0] <= size and value[1] <= size):
 
            if(data[value] != "X"):
                
                if(sub_problems[-1]):
                    
                    if(sub_problems[-1][-1] == oposite_move[key]):
                        
                        continue
                    
                valid_move.append(key)
                
    return valid_move

In [5]:
def maze_solver(data, init_pos):
  
    x,y = init_pos
    spot = data[x,y]
    move_dict = {"up":(x-1,y), "down":(x+1,y), "right":(x,y+1), "left":(x,y-1)}

    sub_problems = [[]]
    sub_problems_pos = [(x,y)]
    sub_problems_sum = [0]

    count = 0
    while (spot != 'G'):

        count+=1
        x,y = sub_problems_pos[-1]
        move_dict = {"up":(x-1,y), "down":(x+1,y), "right":(x,y+1), "left":(x,y-1)}
        valid_move = move_validation(move_dict, sub_problems, len(data)-1)

        # If there is only one valid move, make that move   
        if len(valid_move) == 1:

            x,y = move_dict[valid_move[0]]
            spot = data[x,y]

            sub_problems[-1].append(valid_move[0])
            sub_problems_pos[-1] = (x,y)

            if spot != "G":
                sub_problems_sum[-1] += int(spot)

            #print(sub_problems)

        # If a bifurcation, store current path as a subproblem for each valid move
        elif len(valid_move) > 1: 

            fork_moves = sub_problems[-1].copy()
            fork_pos = sub_problems_pos[-1]
            fork_sum = sub_problems_sum[-1]

            sub_problems.pop()
            sub_problems_pos.pop()
            sub_problems_sum.pop()

            for move in valid_move:

                sub_problems.append(fork_moves + [move])
                sub_problems_pos.append((move_dict[move]))

                spot = data[move_dict[move]]
                sub_problems_sum.append(fork_sum + int(spot))

        elif len(valid_move) == 0:

            if len(sub_problems) == 1:
                print("No path found")
                print(count)
                break

            # If a dead end, backtrack to the last subproblem
            else:

                sub_problems.pop()
                sub_problems_pos.pop()
                x,y = sub_problems_pos[-1]
                continue
                
    return sub_problems_pos[-1], sub_problems_sum[-1]

In [6]:
for path in list(Path().joinpath("data").glob("*.txt")):
    
    data = data_preprocessing(path)
    init_pos = find_init_pos(data)
    final_pos, final_sum = maze_solver(data, init_pos)
        
    print(path)
    print(final_pos)
    print(final_sum)
    print("---------------------------------------------------")

data/maze_101x101.txt
(0, 0)
2622
---------------------------------------------------
data/maze_31x31.txt
(0, 0)
2033
---------------------------------------------------
data/maze_11x11.txt
(0, 0)
191
---------------------------------------------------
